In [1]:
import spacy
import json
from spacy import displacy
from flask import Flask, request, jsonify
from flask_cors import CORS
import pandas as pd
import ast
import requests
import numpy as np
from sklearn.neighbors import NearestNeighbors

In [2]:
app = Flask(__name__)
CORS(app)

In [3]:
@app.route('/resumeparser', methods=['POST'])
def resumeparser():
    record = json.loads(request.data)
    print(record)
    print(request.data)
    nlp = spacy.load("en_core_web_lg")
    ruler = nlp.add_pipe("entity_ruler", before="ner")
    ruler.from_disk("Skill_Ruleset.jsonl")
    patterns = [
        {
            "label": "EMAIL", "pattern": [{"TEXT": {"REGEX": "([^@|\s]+@[^@]+\.[^@|\s]+)"}}]
        },
        {
            "label": "MOBILE", "pattern": [{"TEXT": {"REGEX": "\d{3}[-\.\s]??\d{3}[-\.\s]??\d{4}|\(\d{3}\)\s*\d{3}[-\.\s]??\d{4}|\d{3}[-\.\s]??\d{4}"}}]
        },
        {
            "label": "PERSONNAME", "pattern": [{'POS': 'PROPN'}, {'POS': 'PROPN'}]
        }
    ]
    ruler.add_patterns(patterns)

    text=record["description"]
    
    doc = nlp(text)

    dict={}
    skills=[]
    i=0
    for ent in doc.ents:
        if ent.label_ == 'PERSONNAME' and i == 0:
            dict["personname"] = ent.text
            i = i+1
        if ent.label_ == "EMAIL":
            dict["email"] = ent.text
        if ent.label_ == "MOBILE":
            dict["mobile"] = ent.text
        if ent.label_ == "SKILL":
            skills.append(ent.text)

    skills= [i.capitalize() for i in set([i.lower() for i in skills])]
    dict["skills"] = skills
    print(dict)
    return jsonify(dict)

@app.route('/isalive', methods=['GET'])
def isalive():
    return "Alive!"

@app.route('/bestjob', methods=['POST'])
def bestjob():
    record = json.loads(request.data)
    print(record)
    r = requests.get('https://localhost:5001/api/main/userandjobs/'+record["id"], verify=False)
    item = r.json()
    print("yazdırma")
    print(item)
    items = dict()
    for x in item:
        items[x["id"]] = x["skills"]
        
    df = pd.DataFrame.from_dict(items, orient='index')
    df = pd.get_dummies(df)
    
    withoutUser = df.drop(labels=[record["id"]], axis = 0, inplace=False)
    
    #Creating model and fit
    neigh = NearestNeighbors(n_neighbors=3, radius=0.4)
    neigh.fit(withoutUser.to_numpy())
    
    resultNeighbors = neigh.kneighbors([df.loc[record["id"]].to_numpy()], 3, return_distance=False)
    
    indexes = df.index.values.tolist()
    
    result = []
    for x in resultNeighbors[0]:
        result.append(indexes[x])
    
    finalarray = []
    for x in result:
        headers = {"userId": record["id"]}
        r = requests.get('https://localhost:5001/api/main/workitem/'+x, headers=headers, verify=False)
        item = r.json()
        finalarray.append(item)
        
    return jsonify(finalarray)

In [ ]:
app.run(port=4888)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:4888/ (Press CTRL+C to quit)
127.0.0.1 - - [18/May/2022 19:21:56] "OPTIONS /bestjob HTTP/1.1" 200 -


{'id': '41ad510c-c702-48c1-bae5-393914f46c99'}
yazdırma
[{'id': 'ff1a1caa-20dd-4941-85ba-06233014b68d', 'skills': ['Ai', 'Deployment', 'Design', 'Databases', 'Computer science', 'Azure', 'Redis', 'Software', 'Computer vision', 'Mongodb', 'Docker', 'Python', 'Business', 'Api', 'Helm', 'Postgresql', 'Software engineering', 'Django', 'Linux', 'Kubernetes']}, {'id': '902aaf16-30a9-4113-bf07-c1a98e36c2a8', 'skills': ['Kubernetes', 'Python']}, {'id': 'c3af74f8-3d5a-4194-acbf-36c081f0fc66', 'skills': ['React', 'Javascript', 'Typescript', 'Languages', 'Server', 'Python', 'Css', 'Html', 'Git', 'Linux', 'Libraries']}, {'id': 'a86db4a5-8f22-4d16-b05c-48c9c90d2459', 'skills': ['Business', 'Ai', 'Api', 'Design', 'Security', 'Engineering', 'Java', 'Ml', 'Mobile', 'Software', 'Collaboration']}, {'id': '4eee47a9-2201-4533-a705-e99f79148bba', 'skills': ['Business', 'Design', 'Spring', 'Database', 'Security', 'Java', 'Languages', 'Support', 'Time to market', 'Operating system', 'Software', 'Concurrency'

C:\Users\Burak\anaconda3\lib\site-packages\urllib3\connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\Burak\anaconda3\lib\site-packages\urllib3\connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\Burak\anaconda3\lib\site-packages\urllib3\connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\Burak\anaconda3\lib\site-packages\urllib3\connectionpool.py:981: Insec

{'id': '41ad510c-c702-48c1-bae5-393914f46c99'}
yazdırma
[{'id': 'ff1a1caa-20dd-4941-85ba-06233014b68d', 'skills': ['Ai', 'Deployment', 'Design', 'Databases', 'Computer science', 'Azure', 'Redis', 'Software', 'Computer vision', 'Mongodb', 'Docker', 'Python', 'Business', 'Api', 'Helm', 'Postgresql', 'Software engineering', 'Django', 'Linux', 'Kubernetes']}, {'id': '902aaf16-30a9-4113-bf07-c1a98e36c2a8', 'skills': ['Kubernetes', 'Python']}, {'id': 'c3af74f8-3d5a-4194-acbf-36c081f0fc66', 'skills': ['React', 'Javascript', 'Typescript', 'Languages', 'Server', 'Python', 'Css', 'Html', 'Git', 'Linux', 'Libraries']}, {'id': 'a86db4a5-8f22-4d16-b05c-48c9c90d2459', 'skills': ['Business', 'Ai', 'Api', 'Design', 'Security', 'Engineering', 'Java', 'Ml', 'Mobile', 'Software', 'Collaboration']}, {'id': '4eee47a9-2201-4533-a705-e99f79148bba', 'skills': ['Business', 'Design', 'Spring', 'Database', 'Security', 'Java', 'Languages', 'Support', 'Time to market', 'Operating system', 'Software', 'Concurrency'

...

127.0.0.1 - - [18/May/2022 23:29:33] "OPTIONS /bestjob HTTP/1.1" 200 -


{'id': '3b525a7c-5169-4394-bb8a-1d5747804fa0'}
yazdırma
[{'id': 'ff1a1caa-20dd-4941-85ba-06233014b68d', 'skills': ['Ai', 'Deployment', 'Design', 'Databases', 'Computer science', 'Azure', 'Redis', 'Software', 'Computer vision', 'Mongodb', 'Docker', 'Python', 'Business', 'Api', 'Helm', 'Postgresql', 'Software engineering', 'Django', 'Linux', 'Kubernetes']}, {'id': '902aaf16-30a9-4113-bf07-c1a98e36c2a8', 'skills': ['Kubernetes', 'Python']}, {'id': 'c3af74f8-3d5a-4194-acbf-36c081f0fc66', 'skills': ['React', 'Javascript', 'Typescript', 'Languages', 'Server', 'Python', 'Css', 'Html', 'Git', 'Linux', 'Libraries']}, {'id': 'a86db4a5-8f22-4d16-b05c-48c9c90d2459', 'skills': ['Business', 'Ai', 'Api', 'Design', 'Security', 'Engineering', 'Java', 'Ml', 'Mobile', 'Software', 'Collaboration']}, {'id': '4eee47a9-2201-4533-a705-e99f79148bba', 'skills': ['Business', 'Design', 'Spring', 'Database', 'Security', 'Java', 'Languages', 'Support', 'Time to market', 'Operating system', 'Software', 'Concurrency'

C:\Users\Burak\anaconda3\lib\site-packages\urllib3\connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\Burak\anaconda3\lib\site-packages\urllib3\connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\Burak\anaconda3\lib\site-packages\urllib3\connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\Burak\anaconda3\lib\site-packages\urllib3\connectionpool.py:981: Insec

{'id': '3b525a7c-5169-4394-bb8a-1d5747804fa0'}
yazdırma
[{'id': 'ff1a1caa-20dd-4941-85ba-06233014b68d', 'skills': ['Ai', 'Deployment', 'Design', 'Databases', 'Computer science', 'Azure', 'Redis', 'Software', 'Computer vision', 'Mongodb', 'Docker', 'Python', 'Business', 'Api', 'Helm', 'Postgresql', 'Software engineering', 'Django', 'Linux', 'Kubernetes']}, {'id': '902aaf16-30a9-4113-bf07-c1a98e36c2a8', 'skills': ['Kubernetes', 'Python']}, {'id': 'c3af74f8-3d5a-4194-acbf-36c081f0fc66', 'skills': ['React', 'Javascript', 'Typescript', 'Languages', 'Server', 'Python', 'Css', 'Html', 'Git', 'Linux', 'Libraries']}, {'id': 'a86db4a5-8f22-4d16-b05c-48c9c90d2459', 'skills': ['Business', 'Ai', 'Api', 'Design', 'Security', 'Engineering', 'Java', 'Ml', 'Mobile', 'Software', 'Collaboration']}, {'id': '4eee47a9-2201-4533-a705-e99f79148bba', 'skills': ['Business', 'Design', 'Spring', 'Database', 'Security', 'Java', 'Languages', 'Support', 'Time to market', 'Operating system', 'Software', 'Concurrency'

C:\Users\Burak\anaconda3\lib\site-packages\urllib3\connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\Burak\anaconda3\lib\site-packages\urllib3\connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\Burak\anaconda3\lib\site-packages\urllib3\connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\Burak\anaconda3\lib\site-packages\urllib3\connectionpool.py:981: Insec

In [ ]:
#text="Develop and deliver world-class modern digital solutions for our global operations leveraging latest technologies and design patterns Help translating our internal clients' business requirements into digital solutions to realize our digitalization strategy Collaborate and work closely with business partners, architects and delivery teams throughout the globe and entire solution lifecycle Assist with the maintenance and support of new as well as existing solutions Research new technologies and develop potential scenarios that build on and enhance the existing landscape Qualifications/Experience: Our working language is English. Therefore, excellent command of English (written and verbal) is a must, German would be an asset Bachelor's degree in a related discipline Experience with Java, Spring Framework (Spring MVC, Spring Boot, Spring Data), Angular or VueJs, Javascript, TypeScript, Web Services (SOAP, REST), Microservices Architecture, JPA, Hibernate Working knowledge of one or more of the following is also highly desirable: Docker, Design Patterns (MVVM, Repository, Unit of Work, IoC Container, Dependency Injection, etc.), TDD (xUnit, JUnit), Atlassian products (JIRA Software, Confluence, Bitbucket) Good understanding of project management, system/software development lifecycle (SDLC) techniques and various industry standard agile practices (DevOps, Continuous Delivery, Continuous Integration, Scrum, Kanban) Ability to work in international teams No restriction for international travel"

#doc = nlp(text)
#displacy.render(doc, style="ent", jupyter=True)

In [ ]:
#!python -m spacy download en_core_web_lg

In [ ]:
print("Anurag Asthana\nPune, Maharashtra - Email me on Indeed: indeed.com/r/Anurag-Asthana/ea7451b2bdb6115a\n\n• Looking forward for joining a company where my creative thinking and excellent skills in\napplication development using modern development tools will be utilized.\n\nWilling to relocate: Anywhere\n\nWORK EXPERIENCE\n\nInfosys LTD -  MAHARASHTRA, IN -\n\nApril 2017 to Present\n\nLocation: Pune, India\nClient: Insurance Client from London\nDuration: April 2017 -Till Date\nTechnology/Tools: Microsoft SharePoint 2013, Azure Cognitive Services, Azure Service Bus, Azure\nBlob Storage, .net framework 4.7\n\nResponsibilities:\n\n• Independently worked on designing solution architecture of the project.\n• Lead team from technical front in all the components built\n• Followed Agile development approach to efficiently manage continuous development process\nand incremental requirement changes\n• Design SharePoint components using SharePoint CSOM and .net framework 4.7\n• Design Azure components using azure .net sdks and .net framework 4.7\n• Implement Micro Services architecture using Azure Service Bus\n\nProject Title: SharePoint 2013 Application Development and Enhancement\n\nInfosys LTD -  London -\n\nFebruary 2017 to March 2018\n\nInfosys LTD -  MAHARASHTRA, IN -\n\nAugust 2015 to January 2017\n\nSharePoint Designer\n\nMicrosoft SharePoint -\n\n2017 to 2017\n\n2013, Nintex.\n\nResponsibilities:\n• Investigated and identified requirements via process flows, use cases communicated with\nSubject Matter Experts to define system requirement.\n\nhttps://www.indeed.com/r/Anurag-Asthana/ea7451b2bdb6115a?isid=rex-download&ikw=download-top&co=IN\n\n\n• Suggested improvements to be included into new system for Skills Alignment Portal\n• Followed Agile development approach to efficiently manage continuous development process\nand incremental requirement changes\n• Independently designed, developed and delivered multiple forms and workflows over\nSharePoint 2013 start from scratch using Nintex\n• Designed and Implemented useful components using SharePoint out-of-the-box capabilities and\nSharePoint REST web service\n• Utilized SharePoint APPs technology to create custom enhancements on current sites for\ncustomer satisfaction and efficiency\n• Developed underwriting support applications using asp.net MVC / Web API.\n\nInfosys LTD -  MAHARASHTRA, IN -\n\nDecember 2013 to July 2015\n\nEnterprise IT Support\n\nInfosys LTD -  MAHARASHTRA, IN -\n\nSeptember 2013 to November 2013\n\nLocation: Pune, India\nClient: Oil and Gas Client from USA\nDuration: September 2013 - November 2013\n\nInfosys LTD -  MAHARASHTRA, IN -\n\nJune 2013 to August 2013\n\nLocation: Pune, India\nClient: Oil and Gas Client from USA\nDuration: June 2013 - August 2013\n\nInfosys LTD -  MAHARASHTRA, IN -\n\nFebruary 2013 to May 2013\n\nInfosys LTD -  MAHARASHTRA, IN -\n\nNovember 2012 to January 2013\n\nLocation: Pune, India\nClient: Infosys Ltd.\nDuration: November 2012 - January 2013\n\nSharePoint Designer, Avepoint Docave\n\nMicrosoft SharePoint -\n\n2012 to 2012\n\n6.\n\nResponsibilities:\n\n\n\n• Responsible for creating SharePoint hosted apps.\n• Responsible for developing web services [WEB API's].\n• Responsible for creating ECMA scripts.\n• Responsible for creating console applications using SharePoint Client Object Model.\n• Responsible for migrating SharePoint MOSS 2007 sites to SharePoint 2013 using Avepoint\nDocave.\n• Responsible for developing remote site provisioning mechanism using CSOM\n• Responsible for creating provider hosted apps using asp.net mvc.\n\nProject Title: Operating System\n\nMicrosoft Fast Search, SharePoint Designer\n\nMicrosoft SharePoint -\n\n2010 to 2010\n\nResponsibilities:\n\n• Responsible for maintaining database of the project.\n• Responsible for designing page layouts.\n• Responsible for creating ECMA scripts.\n• Responsible for creating console applications using SharePoint Client Object Model.\n• Responsible for creating SSIS solution to import data from multiple sources using SharePoint\nweb services.\n• Responsible for creating custom SharePoint alerts using SSIS solution.\n• Responsible for creating SSRS solution to create Dashboards.\n• Responsible for creating Stored Procedures and Functions used for various operations.\n• Responsible for customizing list forms.\n• Responsible for creating Search pages using Microsoft FAST search 2010.\n\nMicrosoft SharePoint -\n\n2010 to 2010\n\nResponsibilities:\n\n• Responsible for fixing bugs in SharePoint applications\n• Responsible for automating bug fixing process.\n• Responsible for interacting with clients for understanding business requirements.\n• Responsible for enhancing SharePoint application as per business requirements.\n• Identifying limitations and suggesting appropriate solutions.\n\nMicrosoft InfoPath Designer\n\nMicrosoft SharePoint -\n\n2010 to 2010\n\n2010, Microsoft SharePoint Designer 2010, Microsoft Office 2010.\n\nResponsibilities:\n\n\n\n• Responsible for creating multiple custom lists and document libraries.\n• Responsible for creating and customizing InfoPath forms and associating it with document\nlibraries.\n• Responsible for creating SharePoint Designer Workflows.\n• Responsible for customizing list forms and views.\n• Responsible for creating Excel reports in SharePoint.\n\nProject Title: 5 Day Close Monitoring Tool\n\nTechnology/Tools: Microsoft SharePoint 2010, Microsoft SQL Server 2008 R2, Microsoft Visual\nStudio 2010, Microsoft InfoPath Designer 2010, Microsoft SharePoint Designer 2010, Microsoft\nOffice 2010\nResponsibilities:\n\n• Responsible for creating custom lists, document libraries, SharePoint groups.\n• Responsible for creating custom permission levels in SharePoint and defining permission\npolicies on lists, libraries and web parts.\n• Responsible for maintaining database of the project.\n• Responsible to create SSIS packages.\n• Responsible to create power pivot reports.\n• Responsible for creating dashboards on SharePoint site using Power Pivot library.\n• Responsible for creating tool for generating power pivot reports on multiple servers.\n\nMicrosoft Designer\n\nMicrosoft SharePoint -\n\n2010 to 2010\n\n2010.\nResponsibilities:\n\n• Responsible for creating multiple custom lists and document libraries.\n• Responsible for creating and customizing InfoPath forms and associating it with document\nlibraries.\n• Responsible for creating SharePoint Designer Workflows.\n• Responsible for customizing list forms and views.\n• Responsible for creating custom permission levels and applying it to various lists and libraries.\n\nEDUCATION\n\nBachelor of Technology in Information Technology\n\nGraphic Era University -  Dehra Dun, Uttarakhand\n\nMay 2012\n\nSenior Secondary -  Kashipur, Uttarakhand\n\n2008\n\n\n\nSKILLS\n\nMICROSOFT SHAREPOINT (7 years), SHAREPOINT (7 years), .NET (6 years), MICROSOFT VISUAL\nSTUDIO (6 years), VISUAL STUDIO (6 years)\n\nADDITIONAL INFORMATION\n\nSKILLS\n• Vast knowledge of developing and implementing applications based on client's needs.\n• Professional Experience in creating SharePoint solutions using C#, JavaScript etc.\n• Professional Experience in creating asp.net web applications using MVC, Entity Framework.\n• Professional experience in creating application using Azure micro services architecture\n\nTECHNICAL EXPERTISE\n\nLanguages and Software\nC, C#.net, Asp.net MVC 5, Entity Framework 6, Microsoft Office, Microsoft SharePoint 2010,\nMicrosoft SharePoint 2013, Microsoft, SharePoint Designer, Microsoft Visual Studio 2008,\nMicrosoft Visual Studio 2010, Microsoft Visual Studio 2012, InfoPath, HTML, CSS, JavaScript,\njQuery, SharePoint Apps using MVC and KnockoutJS, Nintex forms and Workflows, Azure\n\nMSBI SQL Server 2008, SQL Server 2012, SSIS, SSRS\nOperating System Windows Family\n\nTOTAL EXPERIENCE\n\nNo. Company Role Duration\n1. Infosys Ltd Technology Analyst 5 Years 11 Months")